# Red neuronal perceptrón multicapa (PM)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk #NLP
nltk.download('stopwords') #NLP
nltk.download('punkt') #NLP
from nltk.corpus import stopwords #NLP
from gensim.utils import simple_preprocess#NLP
from nltk.stem.snowball import SnowballStemmer #NLP
from sklearn.feature_extraction.text import TfidfVectorizer #NLP
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Se procede a hacer limpieza de la data quitando palabras vacias y hacer stem

In [ ]:
data_et = pd.read_excel('/content/drive/MyDrive/Maestria MAD/4 Semestre/Proyecto de tésis/Data sin clasificar.xlsx') #Shamir
data_en = pd.read_excel('/content/drive/MyDrive/Maestria MAD/4 Semestre/Proyecto de tésis/Data clasificada.xlsx') #Shamir

In [ ]:
data_et=data_et[['Calificación','Análisis_x']][0:4000]
data_en= data_en[['Análisis_x','Calificacion_analisis']]

In [ ]:
Texto=data_et['Análisis_x']
Texto1=data_en['Análisis_x']

## Eliminación de ruido

In [ ]:

processed_text0 = []

url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
menciones = '@[\w\-]+'
hashtag = '#[\w\-]+'
#caracteres_especiales = r'\W'
caracter_individual=r'\s+[a-zA-Z]\s+'
caracter_individual_inicio= r'\^[a-zA-Z]\s+'
varios_espacios= r'\s+'
prefijo_b = r'^b\s+'

for t in Texto:
    texto_procesado = t.lower()  #Convertir a minúsculas
    texto_procesado = re.sub(menciones, ' ', texto_procesado)
    texto_procesado = re.sub(hashtag, ' ', texto_procesado)
    texto_procesado = re.sub(url, ' ', texto_procesado)
    #texto_procesado = re.sub(caracteres_especiales, ' ', texto_procesado)
    texto_procesado = re.sub(caracter_individual, ' ', texto_procesado)
    texto_procesado = re.sub(caracter_individual_inicio, ' ', texto_procesado) 
    texto_procesado = re.sub(prefijo_b, '', texto_procesado)
    texto_procesado = re.sub(varios_espacios, ' ', texto_procesado, flags=re.I)
    texto_procesado= re.sub("á",'a',texto_procesado)
    texto_procesado= re.sub("é",'e',texto_procesado)
    texto_procesado= re.sub("í",'i',texto_procesado)
    texto_procesado= re.sub("ó",'o',texto_procesado)
    texto_procesado= re.sub("ú",'u',texto_procesado)
    texto_procesado= re.sub("ñ",'n',texto_procesado)
    texto_procesado= re.sub("'",'',texto_procesado)

    processed_text0.append(texto_procesado)   #agregar a la lista de tweets procesados

processed_text1 = []
for t in Texto1:
    texto_procesado = t.lower()  #Convertir a minúsculas
    texto_procesado = re.sub(menciones, ' ', texto_procesado)
    texto_procesado = re.sub(hashtag, ' ', texto_procesado)
    texto_procesado = re.sub(url, ' ', texto_procesado)
    #texto_procesado = re.sub(caracteres_especiales, ' ', texto_procesado)
    texto_procesado = re.sub(caracter_individual, ' ', texto_procesado)
    texto_procesado = re.sub(caracter_individual_inicio, ' ', texto_procesado) 
    texto_procesado = re.sub(prefijo_b, '', texto_procesado)
    texto_procesado = re.sub(varios_espacios, ' ', texto_procesado, flags=re.I)
    texto_procesado= re.sub("á",'a',texto_procesado)
    texto_procesado= re.sub("é",'e',texto_procesado)
    texto_procesado= re.sub("í",'i',texto_procesado)
    texto_procesado= re.sub("ó",'o',texto_procesado)
    texto_procesado= re.sub("ú",'u',texto_procesado)
    texto_procesado= re.sub("ñ",'n',texto_procesado)
    texto_procesado= re.sub("'",'',texto_procesado)

    processed_text1.append(texto_procesado)   #agregar a la lista de tweets procesados

## Eliminacion de stop words

In [ ]:
stop_words = stopwords.words('spanish')

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
processed_text= processed_text0+processed_text1

In [ ]:
data_words = remove_stopwords(processed_text)

## Stem

In [ ]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    #tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in text:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens


In [ ]:
stemmer = SnowballStemmer("spanish")
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in data_words:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list


In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=20000,
                                 min_df=0.2, stop_words=None,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,2))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(processed_text) #fit the vectorizer to synopses

CPU times: user 27.7 s, sys: 105 ms, total: 27.8 s
Wall time: 30.1 s


In [ ]:
tdidf=tfidf_matrix[:4000].toarray()
tdidf1=tfidf_matrix[4000:].toarray()

In [ ]:
data_et['TDIDF']=[x for x in tdidf]
data_en['TDIDF']=[x for x in tdidf1]

In [ ]:
ent=data_en[['Calificacion_analisis', 'TDIDF']]
pru = data_et['TDIDF']
pru=tf.stack(pru)

In [ ]:
train_labe = to_categorical(ent['Calificacion_analisis'])

In [ ]:
train_labe = tf.stack(train_labe)
test_data = tf.stack(data_en['TDIDF'])

## Aplicar los hiperparametros encontrados

Después de realizar la busqueda de Hyperparametros en busqueda en grilla se tienen en cuenta los siguientes hyperparametro


In [ ]:
data_en['Análisis_x']=processed_text1
data_et['Análisis_x']=processed_text0

In [ ]:
Hyper={'activation1': 'relu',
 'activation2': 'sigmoid',
 'batch_size': 50,
 'dropout_rate': 0.0,
 'epochs': 50,
 'learn_rate': 0.01,
 'neurons1': 150,
 'neurons2': 70}

Se crea función para crear red neuronal 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import metrics

In [ ]:
def create_network(optimizer='rmsprop',neurons1=16, neurons2=16,epochs=50, batch_size=5,
                     learn_rate=0.1,  dropout_rate=0.0, activation1='relu',activation2='relu'):
    model = Sequential()
    model.add(Dense(neurons1,
                    input_shape=(114,),
                    activation=activation1,
                    kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(neurons2, activation=activation2))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss = 'categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy',tf.keras.metrics.Precision()])           
    
    #model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    #            optimizer=optimizer,
    #            metrics=['accuracy'])
    return model

In [ ]:
test_data.shape

TensorShape([4733, 114])

In [ ]:
train_labe.shape

TensorShape([4733, 3])

In [ ]:
Model1 = create_network(**Hyper)
history = Model1.fit(test_data,train_labe,epochs=50,verbose=0)

In [ ]:
predict_prob = Model1.predict(pru)
predict_classes=np.argmax(predict_prob,axis=1)

In [ ]:
tf.keras.metrics.SparseCategoricalAccuracy(
      name='test_accuracy')

In [ ]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'precision'])

In [ ]:
import statistics ##promedio listas
print('accuracy=',statistics.mean(history.history['accuracy']))
print('precision=',statistics.mean(history.history['precision']))

accuracy= 0.8720726829767227
precision= 0.8876558673381806
